In [ ]:
!pip install --upgrade bitsandbytes transformers peft accelerate datasets trl

In [ ]:
import torch, os, multiprocessing
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

#use bf16 and FlashAttention if supported
if torch.cuda.is_bf16_supported():
  os.system('pip install flash_attn')
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

model_name = "microsoft/Phi-3-medium-128k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
assistant_name = "Barbe Noire"
system_prompt = "You are the pirate Barbe Noire. You answer only as Barbe Noire and using pirate speak."
format_system_prompt = "<|start_header_id|>system<|end_header_id|>\n\n"+system_prompt+"<|eot_id|>"
tokenizer.chat_template = "<|begin_of_text|>"+format_system_prompt+"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>"+assistant_name+"<|end_header_id|>\n\n' }}{% endif %}"

ds = load_dataset("Peyton3995/dolly-15k-mistral-pirate")
def process(row):
    messages = [
      {"role": "user", "content": row['instruction']},
      {"role": assistant_name, "content": row['response']},
    ]
    row["text"] = tokenizer.apply_chat_template(messages, tokenize=False)+"<|end_of_text|>"
    return row

ds = ds.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

print(ds['train'][0]['text'])

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, attn_implementation=attn_implementation
)

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        modules_to_save=["lm_head","embed_tokens"],
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj","fc2","fc1"],
)


from trl import SFTConfig

training_arguments = SFTConfig(
        output_dir="./Llama3_8b_Pirate_QLoRA",
        optim="paged_adamw_8bit",
        per_device_train_batch_size=64,
        #gradient_accumulation_steps=4,
        log_level="debug",
        save_strategy="epoch",
        logging_steps=100,
        learning_rate=1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        num_train_epochs=1,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
)

trainer = SFTTrainer(
        model=model,
        train_dataset=ds['train'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()